In [1]:
# generic libraries
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

# project specific custom libraries
from config import *
from simulation import simulate_decay_times
from analysis import binned_maximum_likelihood_fit

In [2]:
#set a seed for numpys random number generator
np.random.seed(42)

In [ ]:
# Exercise part 3 Simple simulation

# Part 0, generate the plot from the N function
t_bounds = [0, 2e-5]
n = 10000

t_space = np.linspace(t_bounds[0], t_bounds[1], n)
t_computed_values = N(t_space, n)

# Part 1, simulate n decay times and create a histogram plot
simulated_decay_points = simulate_decay_times(n, t_bounds[1])

# create the histogram
counts, bins = np.histogram(simulated_decay_points[:, 0] , bins=35)
plt.hist(bins[:-1], bins, weights=counts, color='lightgray', edgecolor='black')
plt.xlabel('Time [s]')
plt.ylabel('Counts')
plt.title(f'Histogram of {n} simulated decay times')
plt.savefig('../Documentation/images/simulated_decay_histogram.png', dpi=750)
plt.close()



results = binned_maximum_likelihood_fit(counts, bins)
print(results)


1991379.1911061353
  message: CONVERGENCE: NORM_OF_PROJECTED_GRADIENT_<=_PGTOL
  success: True
   status: 0
      fun: 1997028.4325915426
        x: [ 1.000e-07  1.000e-05]
      nit: 0
      jac: [ 1.551e+09  4.873e+08]
     nfev: 3
     njev: 1
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>
